In [25]:
# update_db.py
import os
import pandas as pd
import sqlalchemy as sql
from sqlalchemy.sql.schema import Column
from sqlalchemy.sql.sqltypes import String

# print("file path:", os.getcwd()) # root should in ./tdtoolkit_web
raw_root = os.path.join('.','src','app', 'raw')

path = {
    "axo": os.path.join(raw_root, "AXO"),
    "rdl": os.path.join(raw_root, "RDL"),
    "opt": os.path.join(raw_root, "OPT"),
    "rt": os.path.join(raw_root, "RT"),
    "cond": os.path.join(raw_root, "CONDITIONS"),
    "prop": os.path.join(raw_root, "PROPERTY"),
    "ref": os.path.join(raw_root, "REF"),
    "output": "output",
    "db": "database"
}

# need to separate later
def axo_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Could be multiple file
    ─┬─ folder[path]
     ├─ 1-1 + 1-2 + 1-3.csv
     ├─ 1-4.csv
     ...
    """
    df = pd.DataFrame()
    # may wrong, need caution
    loc = [5, 3, 1, 6, 4, 2]
    for cwd, dir_name, file_names in os.walk(path):
        for f in file_names:
            file = os.path.join(cwd, f)
            # get short-id from file name
            # Cause there may be multiple panel in one file, we need to
            # deal with it.
            short_id = f.split(".")[0]
            short_id = [s.strip() for s in short_id.split("+")]
            # multiple each point 6 times cause there are 6 point in one panel
            # maybe I should split this function to another place?
            short_id_6 = [id for id in short_id for _ in range(6)]
            location_6 = loc * len(short_id)
            # Todo: more error format handling
            
            tmp_df = pd.read_csv(file, engine="python", skiprows=27, skipfooter=92)
            tmp_df.insert(loc = 1, column = "short-id", value=short_id_6)
            tmp_df.insert(loc = 2, column = "point", value = location_6)
            # some data has different title(?), so we rename it to make concat well 
            tmp_df.columns = ["Chip No.", "Short-id",  "Point", "x", "y", "cell gap", "top rubbing direct", "twist", "top pre-tilt", "bottom pre-tilt", "rms", "iteration"]
            df = pd.concat([df, tmp_df], ignore_index=True)
    # replace short-id if you have condition table
    if cond.empty != True:
        df = df.rename(columns={"Short-id": "ID", "Chip No.": "LC"})
        df["ID"] = df["ID"].map(dict(cond[["Short-id", "ID"]].values))
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
        # neglect the data that doesn't record
        df = df[~df["ID"].isna()]
        df.columns = ["LC", "ID", "Point", "x", "y", "cell gap", "top rubbing direct", "twist", "top pre-tilt", "bottom pre-tilt", "rms", "iteration", "Project", "Batch"]

    return df

def rdl_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [cell gap].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    if cond.empty != True:
        df = df.rename(columns={"Short-id": "ID"})
        df["ID"] = df["ID"].map(dict(cond[["Short-id", "ID"]].values))
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
        # neglect the data that doesn't record
        df = df[~df["ID"].isna()]
        df.columns = ['ID', 'cell gap', 'LC', "Project", "Batch"]
    return df

def opt_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Could be multiple file
    ─┬─ folder[path]
     ├─ xxxx.csv
     ├─ xxxx.csv
     ...
    """
    df = pd.DataFrame()
    for cwd, dir_name, file_names in os.walk(path):
        for f in file_names:
            if f[0] == ".":
                continue
            if f[0] == "~":
                continue
            file = os.path.join(cwd, f)
            tmp_df = pd.read_csv(file, encoding="ansi").iloc[:,:64]
            # some data preprocessing
            df = pd.concat([df, tmp_df], ignore_index=True)
            
    df.columns = ['Data', 'M_Time', 'ID', 'Point', 'Station', 'Operator', 'Voltage',
        'I.Time', 'AR_T%(⊥)', 'AR_T%(//)', 'LCM_X%', 'LCM_Y%', 'LCM_Z%', 'RX',
        'RY', 'RZ', 'GX', 'GY', 'GZ', 'BX', 'BY', 'BZ', 'WX', 'WY', 'WZ', 'CG%',
        'R_x', 'R_y', 'G_x', 'G_y', 'B_x', 'B_y', 'W_x', 'W_y', 'RX_max',
        'GY_max', 'BZ_max', 'V_RX_max', 'V_GY_max', 'V_BZ_max', "WX'", "WY'",
        "WZ'", "W_x'"," W_x'.1", 'LCM_X%max', 'LCM_Y%max', 'LCM_Z%max',
        'φ_(Ymax)', 'φ_(Ymax).1', 'φ_(Zmax)', 'φ_tol_X', 'φ_tol_Y', 'φ_tol_Z',
        'T0/Tmax_X', 'T0/Tmax_Y', 'T0/Tmax_Z', 'Vcri_X', 'Vcri_Y', 'Vcri_Z',
        'dφ_X', 'dφ_Y', 'dφ_Z', "LC"]
    # voltage == 1 is the wrong rows, need drop
    df = df[df["Voltage"] != 1]
    if cond.empty != True:
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
    return df

def rt_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Could be multiple file
    ─┬─ folder[path]
     ├─ xxxx.txt
     ├─ xxxx.txt
     ...
    """
    df = pd.DataFrame()
    for cwd, dir_name, file_names in os.walk(path):
        for f in file_names:
            if f[0] == ".":
                continue
            if f[0] == "~":
                continue
            file = os.path.join(cwd, f)
            tmp_df = pd.read_table(file, encoding="ansi")
            # some system encoding would go wrong, so I rename here
            # there are some implicit problem, the data should just like this
            tmp_df.columns = ['Date', 'Time', 'ID', 'Point', 'Station', 'Operator', 'cell pos.', 'Target Vpk',
               'Initial Vpk', 'OD_Rise', 'OD_fall', 'Normalized_V', 'Specific_target',
               'Photo Sensor', 'TempSensor', 'Temp', 'Model', 'Rise-mean (10-90)',
               'Rise-stdev (10-90)', 'Fall-mean (10-90)', 'Fall-stdev (10-90)',
               'Rise-mean (5-95)', 'Rise-stdev (5-95)', 'Fall-mean (5-95)',
               'Fall-stdev (5-95)', 'Vcom', 'Flicker', 'Base lv-mean', 'Top lv-mean',
               'WXT (%)', 'BXT (%)', 'WXT_*', 'BXT_*', 'Overshooting or not',
               'Overshooting %', 'TailTime', 'overshooting_peak', 'overshooting_top',
               '(RisePeak-top)/top', '(FallPeak-base)/base', 'delta_peak', 'delta_v',
               'delta_m', 'c_a', 'peak', 'top', 'HLH_(Peak-Top)', 'HLH_area']
            tmp_df = tmp_df[tmp_df["Point"].isin([1, 2, 3, 4, 5, 6, '1', '2', '3', '4', '5', '6'])]
            tmp_df.astype({'Point': 'int32'}, copy=False)
            df = pd.concat([df, tmp_df], ignore_index=True)
    
    if cond.empty != True:
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
    df = df[df["ID"]!="NAN"]
    
    return df

def cond_load(path):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [cond].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    df = df.iloc[:,0:5]
    df.columns = ["ID", "LC", "Short-id", "Project", "Batch"]
    return df

def prop_load(path):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [prop].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    df['Scatter index'] = (df['n_e'] ** 2 - df['n_o'] ** 2) * 3 / df['K11(pN)'] + df['K22(pN)'] + df['K33(pN)']
    df['RT index'] = df['rotation viscosity (γ1)(mPa⋅s)'] / df['K22(pN)']
    return df

def ref_load(path):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [ref].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    return df

# loading data from raw
cond = cond_load(path["cond"])
axo = axo_load(path["axo"], cond)
rdl = rdl_load(path["rdl"], cond)
opt = opt_load(path["opt"], cond)
rt = rt_load(path["rt"], cond)
prop = prop_load(path["prop"])
ref = ref_load(path["ref"])


# writing to database

engine = sql.create_engine('sqlite:///database/test.db', echo=False)
# engine = sql.create_engine('sqlite://', echo=True)

meta = sql.MetaData()
# Model 是否要分出去?
sql.Table(
    "cond", meta,
    sql.Column("LC", sql.String),
    sql.Column("Short-id", sql.String),
    sql.Column("ID", sql.String, unique=True),
    sql.Column("Project", sql.String),
    sql.Column("Batch", sql.String)
)

meta.create_all(engine)

## check constrain
try:
    cond.to_sql("cond", con=engine, if_exists="append", index=False)
    axo.to_sql("axo", con=engine, if_exists="append", index=False)
    rdl.to_sql("rdl", con=engine, if_exists="append", index=False)
    opt.to_sql("opt", con=engine, if_exists="append", index=False)
    rt.to_sql("rt", con=engine, if_exists="append", index=False)
    prop.to_sql("prop", con=engine, if_exists="append", index=False)
    ref.to_sql("ref", con=engine, if_exists="append", index=False)
    print("Database update!")
except:
    print("Something wrong, maybe the ID condition is duplicate. Database keep")

Database update!


In [ ]:
# calculate_summary.py

In [40]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sql
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

In [37]:
# batch = "RD11006008" # X50
batch = "RD11001105" # 1098 TR2

engine = sql.create_engine('sqlite:///database/test.db', echo=False)

cond = pd.read_sql(f"SELECT * FROM cond WHERE batch == \"{batch}\"", engine)
axo = pd.read_sql(f"SELECT * FROM axo WHERE batch == \"{batch}\"", engine)
rdl = pd.read_sql(f"SELECT * FROM rdl WHERE batch == \"{batch}\"", engine)
opt = pd.read_sql(f"SELECT * FROM opt WHERE batch == \"{batch}\"", engine)
rt = pd.read_sql(f"SELECT * FROM rt WHERE batch == \"{batch}\"", engine)
prop = pd.read_sql(f"SELECT * FROM prop", engine)
ref = pd.read_sql(f"SELECT * FROM ref WHERE batch == \"{batch}\"", engine)


In [38]:
len(axo)

0

In [47]:
rt_cell_gap = pd.merge(rt, rdl[["ID", "cell gap"]], how="left", on="ID")

In [50]:
rt_cell_gap["LC"].unique()

array(['LCT-IS-1098', 'SLC19V33L00', 'SLC20V87L00', 'AV369-031-TA',
       'ZIX-7054XX'], dtype=object)

In [41]:
ref_Tr = ref["Tr(ms)"][0]
ref_cell_gap = ref["cell gap(um)"][0]
ref_LC = ref["LC"][0]

# check is there axo data
if len(axo) != 0:
    rt_cell_gap = pd.merge(rt, axo[["ID", "Point", "cell gap"]], how="left", on=["ID", "Point"])
else:
    rt_cell_gap = pd.merge(rt, rdl[["ID", "cell gap"]], how="left", on="ID")
    
df = rt_cell_gap[rt_cell_gap["LC"] == ref_LC].copy()
df["Tr"] = df["Rise-mean (10-90)"]
df["Vop"] = df["Target Vpk"]

df = df.groupby(by=["ID", "Vop", "Point"], as_index=False).mean()

# sns.scatterplot(data=df, x="Vop", y="Tr")

model = {}
model["scalar"] = {}
# Let's try some fasion ML (XD
training_set, test_set = train_test_split(
    df,
    test_size = 0.3,
    random_state = 42
)
X_train = training_set[["Tr", "cell gap"]].to_numpy()
y_train = training_set["Vop"].to_numpy()
X_test = test_set[["Tr", "cell gap"]].to_numpy()
y_test = test_set["Vop"].to_numpy()
model["scalar"]["ref"] = StandardScaler().fit(X_train)
X_train = model["scalar"]["ref"].transform(X_train)
X_test = model["scalar"]["ref"].transform(X_test)
valid_data = model["scalar"]["ref"].transform([[ref_Tr, ref_cell_gap]])
# eXtreme Grandient Boostng Regression
model["Vop_ref_XGBR"] = XGBRegressor(
    n_estimators = 50,
    learning_rate = 0.1,
    max_depth = 3,
    gamma = 0.01,
    reg_lambda = 0.01
)
model["Vop_ref_XGBR"].fit(
    X_train, y_train,
    early_stopping_rounds = 10,
    eval_set = [(X_test, y_test)],
    verbose = False
)
print("R2_train:", model["Vop_ref_XGBR"].score(X_train, y_train))
print("R2_test:", model["Vop_ref_XGBR"].score(X_test, y_test))
ref_Vop = float(model["Vop_ref_XGBR"].predict(valid_data))
print("Vop from Ref[Tr, cell gap]:", ref_Vop)

# Calculate RT, Tf, Tr
df = rt_cell_gap.copy()
df["Vop"] = df["Target Vpk"]
df["RT"] = df["Rise-mean (10-90)"] + df["Fall-mean (10-90)"]
df["Tr"] = df["Rise-mean (10-90)"]
df["Tf"] = df["Fall-mean (10-90)"]
training_set, test_set = train_test_split(
    df,
    test_size = 0.1,
)

model["rt"] = {}
Tr = {}
Tf = {}
RT = {}

for LC in cond["LC"].unique():
    print(LC)
    model["rt"][LC] = {}
    X_train = training_set[training_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    X_test = test_set[test_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    # Standarize & Normalize
    model["scalar"][f'rt_{LC}'] = StandardScaler().fit(X_train)
    X_train = model["scalar"][f'rt_{LC}'].transform(X_train)
    X_test = model["scalar"][f'rt_{LC}'].transform(X_test)
    valid_data = model["scalar"][f'rt_{LC}'].transform([[ref_Vop, ref_cell_gap]])
    for item in ["Tr", "Tf", "RT"]:
        y_train = training_set[training_set["LC"]==LC][item].to_numpy()
        y_test = test_set[test_set["LC"]==LC][item].to_numpy()
        
        model["rt"][LC][f"{item}_XGBR"] = XGBRegressor(
            n_estimators = 3,
            learning_rate = 1,
            max_depth = 3,
            gamma = 1,
            reg_lambda = 1
        ).fit(
            X_train, y_train,
            early_stopping_rounds = 10,
            eval_set = [(X_test, y_test)],
            verbose = False, 
        )
        print(f'RT test {model["rt"][LC][f"{item}_XGBR"].score(X_test, y_test):.2f}')
        ans = float(model["rt"][LC][f"{item}_XGBR"].predict(valid_data))
        print(f"{LC}: {item}: {ans:.2f} ms")


ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [43]:
opt_cell_gap = pd.merge(
    opt,
    axo[["ID", "point", "cell gap"]], 
    left_on=["ID", "Point"], 
    right_on=["ID", "point"], 
    how="left"
)
model["opt"] = {}
df = opt_cell_gap.copy()
df["T%"] = opt_cell_gap.groupby(by=["ID", "Point"])["LCM_Y%"].apply(lambda x: 100*x / float(x.max()))
df["Vop"] = df["Voltage"]/2.0
df["LC%"] = df["LCM_Y%"]
df = df[df["Vop"] > 2]
training_set, test_set = train_test_split(
    df,
    test_size = 0.2,
#     random_state = 42
)
LCp = {}
T = {}
Wx = {}
Wy = {}
for LC in cond["LC"].unique():
    print(LC)
    model["opt"][LC] = {}
    X_train = training_set[training_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    y_train = training_set[training_set["LC"]==LC]["T%"].to_numpy()
    X_test = test_set[test_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    y_test = test_set[test_set["LC"]==LC]["T%"].to_numpy()
    # Standarize & Normalize
    model["scalar"][f'opt_{LC}'] = StandardScaler().fit(X_train)
    X_train = model["scalar"][f'opt_{LC}'].transform(X_train)
    X_test = model["scalar"][f'opt_{LC}'].transform(X_test)
    valid_data = model["scalar"][f'opt_{LC}'].transform([[ref_Vop, ref_cell_gap]])
    # eXtreme Grandient Boostng Regression
    model["opt"][LC]["T_XGBR"] = XGBRegressor(
        n_estimators = 3,
        learning_rate = 1,
        max_depth = 3,
        gamma = 1,
        reg_lambda = 1
    )
    model["opt"][LC]["T_XGBR"].fit(
        X_train, y_train,
        early_stopping_rounds = 10,
        eval_set = [(X_test, y_test)],
        verbose = False, 
    )
#     print("R2_train:", f["opt"][LC]["T_XGBR"].score(X_train, y_train))
    print("R2_test:", model["opt"][LC]["T_XGBR"].score(X_test, y_test))
    T[LC] = float(model["opt"][LC]["T_XGBR"].predict(valid_data))
    print("T%", LC ,"", T[LC])
    y_train = training_set[training_set["LC"]==LC]["W_x"].to_numpy()
    y_test = test_set[test_set["LC"]==LC]["W_x"].to_numpy()
    model["opt"][LC]["Wx_XGBR"] = XGBRegressor(
        n_estimators = 1000,
        learning_rate = 0.01,
        max_depth = 7,
        gamma = 0.001,
        reg_lambda = 0.1,
    )
    model["opt"][LC]["Wx_XGBR"].fit(
        X_train, y_train,
        early_stopping_rounds = 10,
        eval_set = [(X_test, y_test)],
        verbose = False, 
    )
#     print("R2_train:", model["opt"][LC]["Wx_XGBR"].score(X_train, y_train))
    print("R2_test:", model["opt"][LC]["Wx_XGBR"].score(X_test, y_test))
    Wx[LC] = float(model["opt"][LC]["Wx_XGBR"].predict(valid_data))
    print("Wx", LC ,"", Wx[LC])
    
    y_train = training_set[training_set["LC"]==LC]["W_y"].to_numpy()
    y_test = test_set[test_set["LC"]==LC]["W_y"].to_numpy()
    model["opt"][LC]["Wy_XGBR"] = XGBRegressor(
        n_estimators = 100,
        learning_rate = 1,
        max_depth = 5,
        gamma = 0.0001,
        reg_lambda = 0,
    )
    model["opt"][LC]["Wy_XGBR"].fit(
        X_train, y_train,
        early_stopping_rounds = 10,
        eval_set = [(X_test, y_test)],
        verbose = False, 
    )
#     print("R2_train:", model["opt"][LC]["Wy_XGBR"].score(X_train, y_train))
    print("R2_test:", model["opt"][LC]["Wy_XGBR"].score(X_test, y_test))
    Wy[LC] = float(model["opt"][LC]["Wy_XGBR"].predict(valid_data))
    print("Wy", LC ,"", Wy[LC])
    
    y_train = training_set[training_set["LC"]==LC]["LC%"].to_numpy()
    y_test = test_set[test_set["LC"]==LC]["LC%"].to_numpy()
    # eXtreme Grandient Boostng Regression
    model["opt"][LC]["LC_XGBR"] = XGBRegressor(
        n_estimators = 3,
        learning_rate = 1,
        max_depth = 3,
        gamma = 1,
        reg_lambda = 1
    )
    model["opt"][LC]["LC_XGBR"].fit(
        X_train, y_train,
        early_stopping_rounds = 10,
        eval_set = [(X_test, y_test)],
        verbose = False, 
    )
    print("R2_test:", model["opt"][LC]["LC_XGBR"].score(X_test, y_test))
    LCp[LC] = float(model["opt"][LC]["LC_XGBR"].predict(valid_data))
    print("LC%", LC ,"",LCp[LC])   
    print()

LCT-19-580
R2_test: 0.9941543020665929
T% LCT-19-580  92.67816162109375
R2_test: 0.8438619871392379
Wx LCT-19-580  0.26931869983673096
R2_test: 0.9768240466044742
Wy LCT-19-580  0.2690638303756714
R2_test: 0.9312839669830028
LC% LCT-19-580  0.7546943426132202

MOX-1
R2_test: 0.9908828599628081
T% MOX-1  94.47061157226562
R2_test: 0.9571866973857361
Wx MOX-1  0.29174524545669556
R2_test: 0.9939016947751252
Wy MOX-1  0.2935214936733246
R2_test: 0.9297153627060378
LC% MOX-1  0.8463859558105469



In [81]:
# Generate table
summary_table = pd.DataFrame(
    columns=["LC", "platform", "V90", "V95", "V99", "Vmax", "Vop(V)", "Vop_T%", "Δnd(nm)", "Gap(um)", "LC%", "Wx", "Wx_gain", "Wy", "Wy_gain", "u'", "v'", "Ea", "Eb", "ΔEab", "CR", "ΔCR", "T%", "Scatter", "D", "W", "Tr(ms)", "Tf(ms)", "RT(ms)", "G2G(ms)"]
)
# cell gap range
# +- 0.5 um
cell_gap_range = np.linspace(ref_cell_gap-0.5, ref_cell_gap+0.5, 11)
for LC in cond["LC"].unique():
    for cell_gap in cell_gap_range:
        summary_table = summary_table.append({"LC": LC, "Gap(um)": cell_gap}, ignore_index=True)
        # rt
        X = [[ref_Vop, cell_gap]]
        X_scalar = model["scalar"][f'rt_{LC}'].transform(X)
        Tr = model["rt"][LC]["Tr_XGBR"].predict(X_scalar)
        Tf = model["rt"][LC]["Tf_XGBR"].predict(X_scalar)
        RT = model["rt"][LC]["RT_XGBR"].predict(X_scalar)
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "RT(ms)"] = RT[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Tr(ms)"] = Tr[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Tf(ms)"] = Tf[0]
        # opt
        X_scalar = model["scalar"][f'opt_{LC}'].transform(X)
        Wx = model["opt"][LC]["Wx_XGBR"].predict(X_scalar)
        Wy = model["opt"][LC]["Wy_XGBR"].predict(X_scalar)
        T = model["opt"][LC]["T_XGBR"].predict(X_scalar)
        LCp = model["opt"][LC]["LC_XGBR"].predict(X_scalar)
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Wx"] = Wx[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Wy"] = Wy[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "T%"] = T[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "LC%"] = LCp[0] * 100


In [63]:
model['opt']

{'LCT-19-580': {'T_XGBR': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=1, max_delta_step=0, max_depth=3, min_child_weight=1,
               missing=nan, monotone_constraints='()', n_estimators=3, n_jobs=8,
               num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
               scale_pos_weight=1, subsample=1, tree_method='exact',
               validate_parameters=1, verbosity=None),
  'Wx_XGBR': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0.001, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.01, max_delta_step=0, max_depth=7,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=1

In [82]:
summary_table[["LC", "Gap(um)", "Wx", "Wy", "T%", "LC%", "Tr(ms)", "Tf(ms)", "RT(ms)"]]

,LC,Gap(um),Wx,Wy,T%,LC%,Tr(ms),Tf(ms),RT(ms)
0,LCT-19-580,2.0,0.261715,0.252582,92.678162,67.083263,5.605565,5.331964,11.287734
1,LCT-19-580,2.1,0.261715,0.252582,92.678162,67.083263,5.605565,5.331964,11.287734
2,LCT-19-580,2.2,0.261715,0.254415,92.678162,67.083263,5.605565,5.331964,11.287734
3,LCT-19-580,2.3,0.263770,0.257349,92.678162,67.083263,5.834795,5.613353,11.741196
4,LCT-19-580,2.4,0.266355,0.262546,92.678162,75.469434,6.041601,6.161357,12.778079
5,LCT-19-580,2.5,0.269319,0.269064,92.678162,75.469434,6.498720,6.910324,14.034938
6,LCT-19-580,2.6,0.269319,0.268896,92.678162,75.469434,6.895018,7.170544,14.499441
7,LCT-19-580,2.7,0.269319,0.268896,92.678162,75.469434,6.895018,7.170544,14.499441
8,LCT-19-580,2.8,0.269319,0.268896,92.678162,75.469434,6.895018,7.170544,14.499441
9,LCT-19-580,2.9,0.269319,0.268896,92.678162,75.469434,6.895018,7.170544,14.499441


In [75]:
summary_table["LC"] == LC

0    True
Name: LC, dtype: bool

In [77]:
(summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)

0    True
dtype: bool

In [46]:
test_df1 = pd.DataFrame([["A", 2.5]], columns=["LC", "Gap(um)"])
test_df2 = pd.DataFrame([["A", 2.5]], columns=["LC2", "Gap(um)2"])

In [60]:
tmp_df = pd.DataFrame(columns=df.columns)
tmp_df = tmp_df.append({"LC": "A"}, ignore_index=True)

In [61]:
tmp_df.loc[tmp_df.LC == 'A', 'cell gap'] = 2.0
tmp_df

,Data,M_Time,ID,Point,Station,Operator,Voltage,I.Time,AR_T%(⊥),AR_T%(//),...,dφ_Y,dφ_Z,LC,project,batch,point,cell gap,T%,Vop,LC%
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,A,NaN,NaN,NaN,2,NaN,NaN,NaN


In [59]:
tmp_df

,Data,M_Time,ID,Point,Station,Operator,Voltage,I.Time,AR_T%(⊥),AR_T%(//),...,dφ_Y,dφ_Z,LC,project,batch,point,cell gap,T%,Vop,LC%


In [102]:
# reading property table test
# for now it's just messy trash...
pd.read_excel("old_ref/20200924_液晶data sheet (總整理) .xlsx",sheet_name="量產負型AAS",index_col=False)

,產品,*,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Mobile,Mobile.1,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,NB,Unnamed: 22,Unnamed: 23
0,Type,*,AAS,NaN,AAS,NaN,AAS,NaN,AAS,AAS,...,AAS,NaN,AAS,AAS,AAS,NaN,NaN,AAS,NaN,NaN
1,LC Name,*,LCT-13-1386,NaN,LCT-16-1381,NaN,LCT-15-1098,NaN,ZYH-7013XX,LCT-16-1228,...,LCT-16-1381,NaN,LCT-17-1336,LCT-19-580,SLC19V33L00,LCT-16-1200,ZIX-7244XX,LCT-19-1113,NaN,NaN
2,廠區,*,"南廠, T1",NaN,T3,NaN,T1,NaN,L6,T6,...,T3,NaN,T6/T3/T2,T6,NaN,NaN,NaN,NaN,NaN,NaN
3,廠商,NaN,Merck,NaN,Merck,NaN,Merck,NaN,JNC,Merck,...,Merck,NaN,Merck,Merck,SliChem,Merck,JNC,Merck,NaN,NaN
4,廠商量測LC溫度(℃),NaN,20,25.000000,20,25.000000,20,25.000000,25,20,...,20,25.000000,25,25,25,25,25,20,25.000000,30.000000
5,Transition Temp.,Tni(℃),84.5,NaN,84.9,84.900000,85.2,85.200000,80,78.2,...,84.9,84.900000,84.6,79.4,79.9,91.1,NaN,77.8,NaN,NaN
6,NaN,Tcn(℃),-30,-30.000000,NaN,NaN,-30,-30.000000,<-30,-30,...,-30,-30.000000,-30,-30,-30,-20,-15,NaN,NaN,NaN
7,cell Gap (um),NaN,NaN,3.100000,2.8,2.800000,3.1,3.100000,3.2,3.2,...,2.8,2.800000,2.8,3,2.8,3.65,3.65,3.2,3.200000,3.200000
8,d*△n (um),NaN,NaN,0.294810,0.29904,0.293720,0.30256,0.298220,0.32,0.32512,...,0.29904,0.293720,0.3234,0.3201,0.32228,0.309885,NaN,0.30848,0.302080,0.295680
9,Optical Anisotropy (589....,ne,1.5774,1.574100,1.5905,1.587200,1.5794,1.576900,1.586,1.5832,...,1.5905,1.587200,1.6015,1.5925,1.6028,1.563,NaN,1.5776,1.574300,1.571100
